Wykożystaj dane z bazy 'bidevtestserver.database.windows.net'
||
|--|
|SalesLT.Customer|
|SalesLT.ProductModel|
|SalesLT.vProductModelCatalogDescription|
|SalesLT.ProductDescription|
|SalesLT.Product|
|SalesLT.ProductModelProductDescription|
|SalesLT.vProductAndDescription|
|SalesLT.ProductCategory|
|SalesLT.vGetAllCategories|
|SalesLT.Address|
|SalesLT.CustomerAddress|
|SalesLT.SalesOrderDetail|
|SalesLT.SalesOrderHeader|

In [0]:

# INFORMATION_SCHEMA.TABLES

jdbcHostname = "bidevtestserver.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "testdb"

tabela = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:sqlserver://bidevtestserver.database.windows.net:1433;database=testdb") \
  .option("user", "sqladmin") \
  .option("password", "$3bFHs56&o123$") \
  .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("query","SELECT * FROM INFORMATION_SCHEMA.TABLES") \
  .load()

display(tabela)

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
testdb,sys,database_firewall_rules,VIEW
testdb,sys,ipv6_database_firewall_rules,VIEW
testdb,dbo,stg_tablename,BASE TABLE
testdb,SalesLT,Customer,BASE TABLE
testdb,SalesLT,ProductModel,BASE TABLE
testdb,SalesLT,vProductModelCatalogDescription,VIEW
testdb,SalesLT,ProductDescription,BASE TABLE
testdb,SalesLT,Product,BASE TABLE
testdb,SalesLT,ProductModelProductDescription,BASE TABLE
testdb,SalesLT,vProductAndDescription,VIEW


1. Pobierz wszystkie tabele z schematu SalesLt i zapisz lokalnie bez modyfikacji w formacie delta

In [0]:
salesLT = tabela.where("TABLE_SCHEMA == 'SalesLT'")
tab_names = salesLT.select("TABLE_NAME").toPandas()['TABLE_NAME']
print(type(tab_names))

for name in tab_names:
    tab = spark.read \
        .format("jdbc") \
        .option("url",f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase}") \
        .option("user","sqladmin") \
        .option("password","$3bFHs56&o123$") \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("query",f"SELECT * FROM SalesLT.{name}") \
        .load()
    
    tab.write.format("delta").mode("overwrite").saveAsTable(name)

<class 'pandas.core.series.Series'>


Uzycie Nulls, fill, drop, replace, i agg
 * W każdej z tabel sprawdź ile jest nulls w rzędach i kolumnach
 * Użyj funkcji fill żeby dodać wartości nie występujące w kolumnach dla wszystkich tabel z null
 * Użyj funkcji drop żeby usunąć nulle, 
 * wybierz 3 dowolne funkcje agregujące i policz dla TaxAmt, Freight, [SalesLT].[SalesOrderHeader]
 * Użyj tabeli [SalesLT].[Product] i pogrupuj według ProductModelId, Color i ProductCategoryID i wylicz 3 wybrane funkcje agg() 
   - Użyj conajmniej dwóch overloded funkcji agregujących np z (Map)

In [0]:
from pyspark.sql.functions import col, isnan, when, count

tab_names_lower = [name.lower() for name in tab_names]
data_frames = {}

for tab in tab_names_lower:
    filePath = f"dbfs:/user/hive/warehouse/{tab}"
    df = spark.read.format("delta") \
        .option("header","true") \
        .option("inferSchema","true") \
        .load(filePath)
    data_frames[tab] = df

In [0]:
for df in data_frames.values():
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+
|CustomerID|NameStyle|Title|FirstName|MiddleName|LastName|Suffix|CompanyName|SalesPerson|EmailAddress|Phone|PasswordHash|PasswordSalt|rowguid|ModifiedDate|
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+
|         0|        0|    7|        0|       343|       0|   799|          0|          0|           0|    0|           0|           0|      0|           0|
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+

+--------------+----+------------------+-------+------------+
|ProductModelID|Name|CatalogDescription|rowguid|ModifiedDate|
+--------------+----+------------------+-------+------------+
|             0|   0|            

In [0]:
for df in data_frames.values():
    df.na.fill('0', df.columns)

In [0]:
for name, df in data_frames.items():
    data_frames[name] = df.na.drop('all')

In [0]:
from pyspark.sql.functions import avg, min, stddev_pop
soh = data_frames['salesorderheader']

soh.select(avg('Freight')).show()
soh.select(min('TaxAmt')).show()
soh.select(stddev_pop('TaxAmt')).show()

+------------+
|avg(Freight)|
+------------+
|676.11962813|
+------------+

+-----------+
|min(TaxAmt)|
+-----------+
|     3.1163|
+-----------+

+------------------+
|stddev_pop(TaxAmt)|
+------------------+
|2773.8286833211373|
+------------------+



In [0]:
product = data_frames['product']

display(product)

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ThumbNailPhoto,ThumbnailPhotoFileName,rowguid,ModifiedDate
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.3100,1431.5000,58,1016.04,18,6,2002-06-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,43DD68D6-14A4-461F-9069-55309D90EA7E,2008-03-11T10:01:36.827+0000
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.3100,1431.5000,58,1016.04,18,6,2002-06-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,9540FF17-2712-4C90-A3D1-8CE5568B2462,2008-03-11T10:01:36.827+0000
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.9900,null,null,35,33,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712,2008-03-11T10:01:36.827+0000
708,"Sport-100 Helmet, Black",HL-U509,Black,13.0863,34.9900,null,null,35,33,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,A25A44FB-C2DE-4268-958F-110B8D7621E2,2008-03-11T10:01:36.827+0000
709,"Mountain Bike Socks, M",SO-B909-M,White,3.3963,9.5000,M,null,27,18,2005-07-01T00:00:00.000+0000,2006-06-30T00:00:00.000+0000,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,18F95F47-1540-4E02-8F1F-CC1BCB6828D0,2008-03-11T10:01:36.827+0000
710,"Mountain Bike Socks, L",SO-B909-L,White,3.3963,9.5000,L,null,27,18,2005-07-01T00:00:00.000+0000,2006-06-30T00:00:00.000+0000,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,161C035E-21B3-4E14-8E44-AF508F35D80A,2008-03-11T10:01:36.827+0000
711,"Sport-100 Helmet, Blue",HL-U509-B,Blue,13.0863,34.9900,null,null,35,33,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,FD7C0858-4179-48C2-865B-ABD5DFC7BC1D,2008-03-11T10:01:36.827+0000
712,AWC Logo Cap,CA-1098,Multi,6.9223,8.9900,null,null,23,2,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,B9EDE243-A6F4-4629-B1D4-FFE1AEDC6DE7,2008-03-11T10:01:36.827+0000
713,"Long-Sleeve Logo Jersey, S",LJ-0192-S,Multi,38.4923,49.9900,S,null,25,11,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAyAPcAABQlZ/z8/J+fnyxLZ87OzuTk5Dtliqqqqiw0WMrKyvDw8EdXpRAYLsbGxubm5urq6qSkpO7u7uLi4mlmZa+vrxAhNiJai4iVqtLQ5rGxsY2v0zMzNfr6+i5zsiVLcmNwmLK8yHZ1esLCwhhHnfb29lB1rDI= (truncated),awc_jersey_male_small.gif,FD449C82-A259-4FAE-8584-6CA0255FAF68,2008-03-11T10:01:36.827+0000
714,"Long-Sleeve Logo Jersey, M",LJ-0192-M,Multi,38.4923,49.9900,M,null,25,11,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAyAPcAABQlZ/z8/J+fnyxLZ87OzuTk5Dtliqqqqiw0WMrKyvDw8EdXpRAYLsbGxubm5urq6qSkpO7u

In [0]:
from pyspark.sql.functions import skewness, kurtosis

product.groupby('ProductModelId').mean('StandardCost').show()
product.groupby('Color').agg(skewness('Weight')).show()
product.groupby('ProductCategoryId').agg(kurtosis('StandardCost')).show()

+--------------+-----------------+
|ProductModelId|avg(StandardCost)|
+--------------+-----------------+
|            31|     343.64960000|
|            85|       9.34630000|
|            65|      17.37820000|
|            53|      35.95960000|
|            78|     158.53460000|
|           108|       5.77090000|
|            34|    1481.93790000|
|           126|      49.97890000|
|           115|      10.31250000|
|           101|     179.81560000|
|            81|      23.37220000|
|            28|     884.70830000|
|            26|    1541.38733750|
|            27|    1082.51000000|
|            44|      96.79640000|
|           103|      40.62160000|
|            12|      37.12090000|
|            91|      10.84230000|
|           128|      47.28600000|
|            22|     419.77840000|
+--------------+-----------------+
only showing top 20 rows

+------------+--------------------+
|       Color|    skewness(Weight)|
+------------+--------------------+
|        null|   0.7038379

Zadanie 2

In [0]:
from pyspark.sql.types import DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf

def color_short(color):
    if color is None:
        return '-'
    else:
        return color[0].lower()

def round_price(price):
    if price is None:
        return None
    else:
        return round(price, 2)
    
def add(a, b, c):
    if None in [a, b, c]:
        return None
    else:
        return a + b + c

color_short_udf = udf(color_short, StringType())
round_price_udf = udf(round_price, DoubleType())
add_udf = udf(add, IntegerType())

df_1 = product \
    .withColumn('color_short', color_short_udf('Color')) \
    .withColumn('rounded_price', round_price_udf('StandardCost')) \
    .withColumn('sum_of_id', add_udf('ProductID', 'ProductCategoryID', 'ProductModelID')) \
    .select('Color', 'color_short', 'StandardCost', 'rounded_price', 'sum_of_id', 'ProductID', 'ProductCategoryID', 'ProductModelID')
display(df_1)

Color,color_short,StandardCost,rounded_price,sum_of_id,ProductID,ProductCategoryID,ProductModelID
Black,b,1059.3100,null,704,680,18,6
Red,r,1059.3100,null,730,706,18,6
Red,r,13.0863,null,775,707,35,33
Black,b,13.0863,null,776,708,35,33
White,w,3.3963,null,754,709,27,18
White,w,3.3963,null,755,710,27,18
Blue,b,13.0863,null,779,711,35,33
Multi,m,6.9223,null,737,712,23,2
Multi,m,38.4923,null,749,713,25,11
Multi,m,38.4923,null,750,714,25,11


In [0]:
product.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- StandardCost: decimal(19,4) (nullable = true)
 |-- ListPrice: decimal(19,4) (nullable = true)
 |-- Size: string (nullable = true)
 |-- Weight: decimal(8,2) (nullable = true)
 |-- ProductCategoryID: integer (nullable = true)
 |-- ProductModelID: integer (nullable = true)
 |-- SellStartDate: timestamp (nullable = true)
 |-- SellEndDate: timestamp (nullable = true)
 |-- DiscontinuedDate: timestamp (nullable = true)
 |-- ThumbNailPhoto: binary (nullable = true)
 |-- ThumbnailPhotoFileName: string (nullable = true)
 |-- rowguid: string (nullable = true)
 |-- ModifiedDate: timestamp (nullable = true)

